# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import *

In [2]:
# load data from database
engine = create_engine('sqlite:///etl_pipeline.db')
df = pd.read_sql('select * from message_1', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
Y = df.iloc[:,4:39]

In [4]:
X.value_counts()

#NAME?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    4
HELP THE EARTHQUAKE VICTIMS IN HAITI http tinyurl.com yk3bspe links to many resources where you can make donations. Broke? donate time                                                                                                                                                                                                                                                                                                                                                                    2
Me and my family

In [5]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather'],
      dtype='object')

In [6]:
# To understand what each column is having as values and we can see that column 0 i.e 'related' is having [0,1,2] so we have to replace all the 2 values to 1 using replace() in the next cell!
for i in range(35):
    print('column: ',i,Y.iloc[:,i].unique())

column:  0 [1 0 2]
column:  1 [0 1]
column:  2 [0 1]
column:  3 [0 1]
column:  4 [0 1]
column:  5 [0 1]
column:  6 [0 1]
column:  7 [0 1]
column:  8 [0 1]
column:  9 [0]
column:  10 [0 1]
column:  11 [0 1]
column:  12 [0 1]
column:  13 [0 1]
column:  14 [0 1]
column:  15 [0 1]
column:  16 [0 1]
column:  17 [0 1]
column:  18 [0 1]
column:  19 [0 1]
column:  20 [0 1]
column:  21 [0 1]
column:  22 [0 1]
column:  23 [0 1]
column:  24 [0 1]
column:  25 [0 1]
column:  26 [0 1]
column:  27 [0 1]
column:  28 [0 1]
column:  29 [0 1]
column:  30 [0 1]
column:  31 [0 1]
column:  32 [0 1]
column:  33 [0 1]
column:  34 [0 1]


In [7]:
Y['related'].replace(2,1,inplace=True)

In [8]:
Y['related'].unique()

array([1, 0])

### 2. Write a tokenization function to process your text data

In [9]:
import nltk
import re
nltk.download(['punkt','wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [10]:
def tokenize(text):
    text=re.sub('[^\w\s]','',text)
    tokens=word_tokenize(text)
    lemmatizer=WordNetLemmatizer()
    
    tokens_cleaned=[]
    for token in tokens:
        clean_token=lemmatizer.lemmatize(token).lower().strip()
        tokens_cleaned.append(clean_token)
    
    return tokens_cleaned

In [11]:
# To understand how our tokenize function works is given by:
for message in X[:2]:
    print(message)
    print(tokenize(message))
    print()

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti']

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over']



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

In [13]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y)
pipeline.fit(X_train,Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
Y_pred=pipeline.predict(X_test)
Y_pred_df=pd.DataFrame(Y_pred,columns=Y_test.columns)

In [16]:
Y_pred_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
evaluation = {}
for column in Y_test.columns:
    evaluation[column] = []
    evaluation[column].append(precision_score(Y_test[column], Y_pred_df[column]))
    evaluation[column].append(recall_score(Y_test[column], Y_pred_df[column]))
    evaluation[column].append(f1_score(Y_test[column], Y_pred_df[column]))

pd.DataFrame(evaluation)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather
0,0.827847,0.829659,0.0,0.750680,0.640625,0.666667,0.777778,0.0,0.800000,0.0,...,0.0,0.0,0.200000,0.827411,0.858586,0.759091,1.000000,0.884259,0.833333,0.785714
1,0.933320,0.361572,0.0,0.507545,0.076636,0.074074,0.037634,0.0,0.086957,0.0,...,0.0,0.0,0.003521,0.542129,0.151246,0.282095,0.013699,0.626230,0.072993,0.033333
2,0.877425,0.503650,0.0,0.605621,0.136895,0.133333,0.071795,0.0,0.156863,0.0,...,0.0,0.0,0.006920,0.655057,0.257186,0.411330,0.027027,0.733205,0.134228,0.063953


In [18]:
# Let's generate the classification report:
from sklearn.metrics import classification_report
for i in range(35):
    print("Precision, Recall, F1 Score for {} is:".format(Y_test.columns[i]))
    print(classification_report(Y_test.iloc[:,i], Y_pred[:,i]))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision, Recall, F1 Score for related is:
             precision    recall  f1-score   support

          0       0.62      0.35      0.45      1515
          1       0.83      0.93      0.88      5039

avg / total       0.78      0.80      0.78      6554

Precision, Recall, F1 Score for request is:
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5409
          1       0.83      0.36      0.50      1145

avg / total       0.87      0.88      0.85      6554

Precision, Recall, F1 Score for offer is:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6524
          1       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6554

Precision, Recall, F1 Score for aid_related is:
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      3837
          1       0.75      0.51      0.61      2717

avg / total     

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
from sklearn.model_selection import GridSearchCV

parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'tfidf__use_idf': (True, False)
    }

cv = GridSearchCV(pipeline,param_grid=parameters)

In [ ]:
cv.fit(X_train, Y_train)
Y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
print('ACCURACY:',(Y_pred==Y_test).mean())

ACCURACY: related                   0.799512
request                   0.875496
offer                     0.995423
aid_related               0.725969
medical_help              0.921117
medical_products          0.952395
search_and_rescue         0.972383
security                  0.982148
military                  0.967196
child_alone               1.000000
water                     0.954379
food                      0.924168
shelter                   0.934849
clothing                  0.986573
money                     0.977266
missing_people            0.988099
refugees                  0.966128
death                     0.957888
other_aid                 0.865273
infrastructure_related    0.933628
transport                 0.955752
buildings                 0.952090
electricity               0.980623
tools                     0.994202
hospitals                 0.989777
shops                     0.995423
aid_centers               0.985658
other_infrastructure      0.956210
weather_re

In [23]:
for i in range(35):
    print("Precision, Recall, F1 Score for {}".format(Y_test.columns[i]))
    print(classification_report(Y_test.iloc[:,i], Y_pred[:,i]))

Precision, Recall, F1 Score for related
             precision    recall  f1-score   support

          0       0.62      0.35      0.45      1515
          1       0.83      0.93      0.88      5039

avg / total       0.78      0.80      0.78      6554

Precision, Recall, F1 Score for request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5409
          1       0.83      0.36      0.50      1145

avg / total       0.87      0.88      0.85      6554

Precision, Recall, F1 Score for offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6524
          1       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6554

Precision, Recall, F1 Score for aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      3837
          1       0.75      0.51      0.61      2717

avg / total       0.73      0.73

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [24]:
# I'm trying another ensemble classifier that is AdaBoostClassifier which is a sequential ensembling model
from sklearn.ensemble import AdaBoostClassifier
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])


In [25]:
pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)

In [26]:
from sklearn.metrics import classification_report
for i in range(35):
    print("Precision, Recall, F1 Score for {}".format(Y_test.columns[i]))
    print(classification_report(Y_test.iloc[:,i], Y_pred[:,i]))

Precision, Recall, F1 Score for related
             precision    recall  f1-score   support

          0       0.64      0.35      0.45      1515
          1       0.83      0.94      0.88      5039

avg / total       0.78      0.80      0.78      6554

Precision, Recall, F1 Score for request
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5409
          1       0.78      0.52      0.62      1145

avg / total       0.88      0.89      0.88      6554

Precision, Recall, F1 Score for offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6524
          1       0.00      0.00      0.00        30

avg / total       0.99      0.99      0.99      6554

Precision, Recall, F1 Score for aid_related
             precision    recall  f1-score   support

          0       0.74      0.86      0.80      3837
          1       0.75      0.58      0.65      2717

avg / total       0.74      0.74

### 9. Export your model as a pickle file

In [27]:
import pickle
filename='final_model.pkl'
pickle.dump(pipeline,open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.